In [0]:
# --- PARAMETERS (set by ADF) ---
dbutils.widgets.text("db", "mining")                   # database name
dbutils.widgets.text("silver_table", "silver_flotation")
DB = dbutils.widgets.get("db")
SILVER = f"{DB}.{dbutils.widgets.get('silver_table')}"

from pyspark.sql import functions as F, types as T

# 1) Load Bronze
bronze = spark.table(f"{DB}.bronze_flotation")

In [0]:
# 2) Parse timestamp and cast numerics
df = bronze.withColumn("ts", F.to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))
numeric_cols = [c for c in df.columns if c not in ("date","ts")]
for c in numeric_cols:
    df = (df.withColumn(c, F.when(F.col(c).isNull(), None).otherwise(F.trim(F.col(c))))
            .withColumn(c, F.regexp_replace(F.col(c), ",", "."))
            .withColumn(c, F.regexp_replace(F.col(c), r"[^0-9\.\-]", ""))
            .withColumn(c, F.col(c).cast(T.DoubleType())))

In [0]:
# 3) Save Silver
(df.select(["ts"] + numeric_cols)
   .write.format("delta")
   .mode("overwrite")
   .option("overwriteSchema","true")
   .saveAsTable(SILVER))

print(f"✅ Silver built: {SILVER}")